**Importing necessary libraries**

In [1]:
import pandas as pd
import numpy as np
import os
import cv2

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

**Loading model and cascade classifier to detect face**

In [2]:
model=load_model('best-imagenet')

In [3]:
face_clsfr=cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

**Reading video , detecting face and preprocessing it accordingly to predict till escape button is given as input**

In [4]:
source=cv2.VideoCapture(0)

labels_dict={0:'NO MASK',1:'MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}  #0->red (no mask) , 1->green (mask)

while(True):

    ret,img=source.read()
    image=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    faces=face_clsfr.detectMultiScale(image,1.3,5)  

    for x,y,w,h in faces:
    
        face_img=image[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(224,224))    
        face = img_to_array(resized)             
        face = preprocess_input(face)
        faces = np.array(faces, dtype="float32")
        face = np.expand_dims(face, axis=0)
        result=model.predict(face)        

        y_hat=np.argmax(result,axis=1)[0]       
        
        label = "{}: {:.2f}%".format(labels_dict[y_hat], np.max(result,axis=1).item()*100)        
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[y_hat],2)
        cv2.rectangle(img,(x,y),(x+w,y),color_dict[y_hat],-1)
        cv2.putText(img, label, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
  
        
    cv2.imshow('Live face mask detector',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()